In [2]:
import torch

In [170]:
class GAN(torch.nn.Module):
    
    def __init__(self, convtrans_channel=[6,3,1], conv_channel=[1,3,6,1],z_dim=8,channel_input=8):
        
        super(type(self), self).__init__()
        self.z_dim=z_dim
        self.batch=32
        self.channel=channel_input
        
        ##capas generadoras
        self.generative_layers = torch.nn.Sequential(
                                  torch.nn.ConvTranspose2d(self.channel,convtrans_channel[0],4,stride=2,padding=2 ),
                                  torch.nn.BatchNorm2d(convtrans_channel[0]),
                                  torch.nn.ReLU(),
                                  torch.nn.ConvTranspose2d(convtrans_channel[0],convtrans_channel[1],5,padding=4),
                                  torch.nn.BatchNorm2d(convtrans_channel[1]),
                                  torch.nn.ReLU(),
                                  torch.nn.ConvTranspose2d(convtrans_channel[1],convtrans_channel[2],3,stride=2),
                                  torch.nn.Tanh(),
                                  
            
                                )
        self.discriminator_layers= torch.nn.Sequential(
                                   torch.nn.Conv2d(conv_channel[0],conv_channel[1],5,stride=2,padding=1),
                                   torch.nn.LeakyReLU(),
                                   torch.nn.Conv2d(conv_channel[1],conv_channel[2],5,stride=3,padding=1),
                                   torch.nn.BatchNorm2d(conv_channel[2]),
                                   torch.nn.LeakyReLU(),
                                   torch.nn.Conv2d(conv_channel[2],conv_channel[3],5,stride=2,padding=1),
                                   torch.nn.Sigmoid(),                                
        
                                )
        
        
        self.activation=torch.nn.ReLU()
        
    
    def generador(self, z):
        x_falso=self.generative_layers(z)
        print(x_falso.shape)
        return x_falso
        
    def discriminador(self,supuesto_x):
        supuesto_x=self.discriminator_layers(supuesto_x)
        supuesto_x=supuesto_x.view(-1,1)
        return supuesto_x
    
    def sample(self):
        z=torch.randn(self.batch,self.channel,self.z_dim,self.z_dim)
        x_falso=self.generador(z)
        return z
    
    def forward(self):
        z=self.sample()
        x_falso=self.generador(z)
        ezreal=self.discriminador(x_falso)
        print("ezreal",ezreal)
        print(x_falso.shape)
    


In [171]:
model=GAN()

In [172]:
model.forward()

torch.Size([32, 1, 21, 21])
torch.Size([32, 1, 21, 21])
ezreal tensor([[0.5269],
        [0.5595],
        [0.4718],
        [0.5822],
        [0.5390],
        [0.5509],
        [0.5272],
        [0.5264],
        [0.5235],
        [0.4889],
        [0.5320],
        [0.4788],
        [0.5184],
        [0.5616],
        [0.5589],
        [0.5588],
        [0.5487],
        [0.5603],
        [0.4694],
        [0.5217],
        [0.4980],
        [0.5124],
        [0.5633],
        [0.5878],
        [0.6256],
        [0.4985],
        [0.5209],
        [0.4371],
        [0.5133],
        [0.5448],
        [0.5323],
        [0.4866]], grad_fn=<ViewBackward>)
torch.Size([32, 1, 21, 21])
